<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/LPS%20Training%20-%20Exercise%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LPS hands-on training session: CoMet Toolkit: Uncertainties made easy**

#Exercise 2: Multi-Dimension Datasets

## Objectives

In this exercise we will cover:

* How to use [obsarray](https://obsarray.readthedocs.io/en/latest/) to store error-correlation information for multi-dimensional measurement datasets - such as from Earth Observation.
* Propagating uncertainties from these datasets through measurement functions using [punpy](https://punpy.readthedocs.io/en/latest/).

## Step 1 - Environment Setup

As in Exercise 1, we start by install and importing the required CoMet Toolkit packages --- [obsarray](https://obsarray.readthedocs.io/en/latest/) and [punpy](https://punpy.readthedocs.io/en/latest/).

In [ ]:
# Install & import CoMet Toolkit packages
!pip install obsarray>=1.0.1
!pip install punpy>=1.0.4

import obsarray
import punpy
import xarray as xr

CoMet Toolkit's [**obsarray**](https://obsarray.readthedocs.io/en/latest/) package is an extension to the popular [xarray](https://docs.xarray.dev/en/stable/) package.

[`xarray.Dataset`](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#dataset)'s are objects in python that represent of the data model from the [netCDF file format](https://www.unidata.ucar.edu/software/netcdf/). **obsarray** allows you to assign uncertainties to variables in xarray Datasets, with their associated error-correlation.

This is achieved by using the CoMet Toolkit's [UNC Specification](https://comet-toolkit.github.io/unc_website/) metadata standard for uncertainty. So such objects are portable, and can be stored to and from disc.


## Step 2 - Interacting with a Measurement Dataset using obsarray

In this step of the exercise, we will explore how to define and interact with the *uncertainty variables* (i.e., uncertainty components) associated with *observation variables* in measurement datasets using obsarray.

Our example will be a multi-spectral dataset of Level 1 [Brightness Temperatures](https://en.wikipedia.org/wiki/Brightness_temperature) from the AVHRR sensor on MetOp-A. This [dataset](https://catalogue.ceda.ac.uk/uuid/14a8098d70714cc1bf38f9dbcb82e5ed/) was created as part of the [FIDUCEO](https://research.reading.ac.uk/fiduceo/) project.

Here we open an extract from [netCDF](https://www.unidata.ucar.edu/software/netcdf/), which has an observation variable with the following dimensions:

* $x$, along track -- 400 pixels at 1 km resolution
* $y$, across track -- 400 pixels at 1 km resolution
* $band$, spectral bands -- 2 bands centred on $\sim$11 and 12 $\mu$m


In [ ]:
# open xarray.Dataset from netCDF file
dataset_path = "avhrr_dataset.nc"
avhrr_ds = xr.open_dataset(dataset_path)

# inspect dataset
print(avhrr_ds)
avhrr_ds["brightness_temperature"][0].plot()

After import, **obsarray** functionality is accessed throught the `unc` "[accessor](https://docs.xarray.dev/en/stable/internals/extending-xarray.html)" -- which looks like a new [link text](https://)method that becomes available on xarray Datasets.

We can use this to [assign an *uncertainty variable*](https://obsarray.readthedocs.io/en/latest/content/user/unc_accessor.html#adding-removing-uncertainty-components) to the `brightness_temperature`, in a very similar way to adding a normal variable to an xarray Dataset:

In [ ]:
# add an uncertainty component associated with noise error to the brightness temperature
avhrr_ds.unc["brightness_temperature"]["u_noise"] = ()

print("Dataset Variables: " + str(avhrr_ds.keys())

Uncertainty variables have an assocaited error-correlation structure -- since we didn't define this for `u_noise` it is assumed to be random (i.e., errors are uncorrelated between pixels.

Next let's add a calibration uncertainty component, `u_cal`, with a more complicated error-correlation structure using the `err_corr` attribute. This uses the [error-correlation parameterisations](https://comet-toolkit.github.io/unc_website/specification/draft-v0.1/unc_specification.html#appendix-a-error-correlation-parameterisations) defined by the UNC Specfication.

Let's set errors associated with `u_cal` to be systematic (i.e., the same/common) in the `x` and `y` dimension and defined by a custom matrix in the `band` dimension.

In [ ]:
# create cross-channel error-correlation matrix
chl_err_corr_matrix = np.array([[1.0, 0.7],[0.7, 1.0]])
avhrr_ds["chl_err_corr_matrix"] = (("band1", "band2"), chl_err_corr_matrix)

# use this to define error-correlation parameterisation attribute
err_corr_def = [
    # fully systematic in the x and y dimension
    {
        "dim": ["x", "y"],
        "form": "systematic",
        "params": [],
        "units": []
    },
    # defined by err-corr matrix var in band dimension
    {
        "dim": ["band"],
        "form": "err_corr_matrix",
        "params": ["chl_err_corr_matrix"],  # defines link to err-corr matrix var
        "units": []
    }
]

# define u_cal values - set as 5%
u_cal_values = avhrr_ds["brightness_temperature"].values * 0.05

# add an uncertainty component associated with calibration error to the brightness temperature
avhrr_ds.unc["brightness_temperature"]["u_cal"] = (["x", "y", "band"], u_cal_values, {"err_corr": err_corr_def})

We can now interface with this information

# Step 3 - Uncertainty Propagation

Thermal Infrared multi-spectral data, like our example AVHRR dataset, is commonly used to evaluate sea or land surface temperature (SST or LST). SST/LST retriavals account for the atmosphere to evaluate the surface temperature from the top of atmosphere brightness temperature.

A widely approach for this is called the "split window" method. A simplified form of this algorithm can be defined as,

$SST = a T_{11} + b T_{12}$

where:
* $T_{11}$​ is the brightness temperature in the 11 μm channel
* $T_{12}$​ is the brightness temperature in the 12 μm channel
* $a$ & $b$ are empirically derived retrieval coefficients

##Exercise

Using what we learned in [Exercise 1](https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/LPS_training_exercise1.ipynb), create a measurement function to apply the SST retrieval to our AVHRR dataset and propagate the uncertainties using punpy.

In [ ]:
# Enter your code here

# *Extension* - Propagating Dataset Uncertainties with the `MeasurementFunction` Class

**punpy**'s [`MeasurementFunction`](https://punpy.readthedocs.io/en/latest/content/punpy_digital_effects_table.html#measurement-function) class enables a much simpler method for propagating the uncertainties of measurement datasets defined using **obsarray**. It is an alternative interface to the punpy propagation functions we used in Step 3.

For this approach instead defining the measurement function as a python function, we define a measurement function class which should be a subclass of the punpy `MeasurementFunction` class. We can then use the method `propagate_ds` to propagate all dataset uncertainties in one go!

In [ ]:
class SplitWindowSST(punpy.MeasurementFunction):
  def meas_function(self, BT):
  """
  Split window method retrieval measurement function
  """

  return a * BT[0,:,:] + b * BT[1,:,:]

  # define variable names for output dataset
  def get_measurand_name_and_unit(self):
    return "sst", "K"

  # associate input dataset names with meas_function arguments
  def get_argument_names(self):
    return ["brightness_temperature"]


# create punpy propagation object
prop = MCPropagation(1000, dtype="float32", verbose=False, parallel_cores=4)

# instatiate measurement function object with prop
gl = IdealGasLaw(prop=prop)

# run uncertainty propagatoin
ds_y = gl.propagate_ds(ds_pres, ds_nmol, ds_temp)

# Exercise

Adapt the `MeasurementFunction` class approach above to include uncertainty on the parameters $a$ and $b$.

In [ ]:
# Enter your code here